In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle

In [2]:
from aerb_unv_reader import UNV

In [3]:
from material_prop_reader import material_section_read

In [4]:
with open('EPR_mat.pickle', 'rb') as handle:
    EPR_mat = pickle.load(handle)
with open('GROUPS.pickle', 'rb') as handle:
    GROUPS = pickle.load(handle)
with open('ELEM_GROUPS.pickle', 'rb') as handle:
    ELEM_GROUPS = pickle.load(handle)
with open('NODE_GROUPS.pickle', 'rb') as handle:
    NODE_GROUPS = pickle.load(handle)
with open('MATERIALS.pickle', 'rb') as handle:
    MATERIALS = pickle.load(handle)
with open('ELEMENT_MATERIAL_INX.pickle', 'rb') as handle:
    ELEMENT_MATERIAL_INX =  pickle.load(handle)
with open('Unique_MATERIALS.pickle', 'rb') as handle:
    Unique_MATERIALS = pickle.load(handle)
with open('Unique_MATERIALS.pickle', 'rb') as handle:
    Unique_MATERIALS = pickle.load(handle)
with open('Spring.pickle', 'rb') as handle:
    Spring = pickle.load(handle)

with open('TRUSS.pickle', 'rb') as handle:
    TRUSS = pickle.load(handle)
with open('TRUSS_ELIST.pickle', 'rb') as handle:
    TRUSS_ELIST = pickle.load(handle)

In [5]:
EPR_geo = UNV('InputsV2/PHASE_3_STAGE_1_INPUT_v2/02_Model/CRUAS_NPP.unv')

# Beam Elements

In [6]:
## READ ALL ELEMENTS
ELEMENTS = {}
for key,value in EPR_geo.fem['elements'][21].items():
    ELEMENTS[key] = {'type':21, 'connectivity':value[-2:]}

In [7]:
def beam_section_exists(section,BEAM_SECTIONS):
    for key, sec in BEAM_SECTIONS.items():
        if sec['type'] == section['type']:
            norm1 = np.linalg.norm(np.array(sec['parameters'])-np.array(section['parameters']))
            if norm1 <= 0.01:
                return key
    return None

In [8]:
ii = 1
BEAM_SECTIONS = {}
for key, beam in EPR_mat.section_details['beam'].items():
    chkk = beam_section_exists(beam, BEAM_SECTIONS)
    if not(chkk):
        nname = 'SEC{0:03d}'.format(ii)
        ii = ii + 1
        BEAM_SECTIONS[nname] = {'type': beam['type'],'material':{},\
                                'parameters': beam['parameters'],\
                                'mesh_groups':[key], 'elements':[]}
    else:
        BEAM_SECTIONS[chkk]['mesh_groups'].append(key)

In [9]:
BEAM_MAST_LIST = []
for key, bm in BEAM_SECTIONS.items():
    for grp in bm['mesh_groups']:
        BEAM_SECTIONS[key]['elements'] = BEAM_SECTIONS[key]['elements'] + list(ELEM_GROUPS[grp])
        BEAM_MAST_LIST = BEAM_MAST_LIST + list(ELEM_GROUPS[grp])
BEAM_MAST_LIST = sorted(set(BEAM_MAST_LIST))
BEAM_MAST_LIST_DF = pd.DataFrame(data=None, index=list(BEAM_MAST_LIST), columns=['SECTION','MATERIALS','ORIENTATION'])

In [10]:
BEAM_MAST_LIST_DF

,SECTION,MATERIALS,ORIENTATION
5438,NaN,NaN,NaN
5439,NaN,NaN,NaN
5440,NaN,NaN,NaN
5441,NaN,NaN,NaN
5442,NaN,NaN,NaN
...,...,...,...
13025,NaN,NaN,NaN
13026,NaN,NaN,NaN
13027,NaN,NaN,NaN
13028,NaN,NaN,NaN


In [11]:
for key, bm in BEAM_SECTIONS.items():
    BEAM_SECTIONS[key]['elements'] = sorted(set(BEAM_SECTIONS[key]['elements']))
    mat_list = list(ELEMENT_MATERIAL_INX.loc[bm['elements'],'MATERIALS'].unique())
    tempDB = ELEMENT_MATERIAL_INX.loc[bm['elements']]
    for mm in mat_list:
        BEAM_SECTIONS[key]['material'][mm] = list(tempDB.index[tempDB['MATERIALS'] == mm])
        for eid in BEAM_SECTIONS[key]['material'][mm]:
            BEAM_MAST_LIST_DF.loc[eid,'SECTION'] = key
            BEAM_MAST_LIST_DF.loc[eid,'MATERIALS'] = mm

In [12]:
with open('BEAM_SECTIONS.pickle', 'wb') as handle:
    pickle.dump(BEAM_SECTIONS, handle, protocol=1)

In [13]:
## Finding Unique Beam Orientations
def beam_orientation_exists(orient,BEAM_ORIENTATION):
    for key, sec in BEAM_ORIENTATION.items():
        norm1 = np.dot(sec['parameters'],orient)
        if norm1 > 0.99:
            return key
    return None

In [14]:
### Finding Unique Beam Orientation
ii = 1
BEAM_ORIENT = {}
BEAM_ORIENT_ONLY_BEAM = {}
for key, beam in EPR_mat.beam_orientation.items():
    vector = np.array([round(float(dd),2) for dd in beam])
    chkk = beam_orientation_exists(vector, BEAM_ORIENT)
    if not(chkk):
        nname = 'ORT{0:03d}'.format(ii)
        ii = ii + 1
        BEAM_ORIENT[nname] = {'parameters': vector,\
                                'mesh_groups':[key], 'elements':[]}
        BEAM_ORIENT_ONLY_BEAM[nname] = {'parameters': vector,\
                                'mesh_groups':[key], 'elements':[]}
    else:
        BEAM_ORIENT[chkk]['mesh_groups'].append(key)
        BEAM_ORIENT_ONLY_BEAM[chkk]['mesh_groups'].append(key)

In [15]:
for key, bm in BEAM_ORIENT.items():
    for grp in bm['mesh_groups']:
        BEAM_ORIENT[key]['elements'] = BEAM_ORIENT[key]['elements'] + list(ELEM_GROUPS[grp])
        for eid in BEAM_ORIENT[key]['elements']:
             BEAM_MAST_LIST_DF.loc[eid,'ORIENTATION'] = key

In [16]:
BEAM_MAST_LIST_DF2 = BEAM_MAST_LIST_DF.loc[BEAM_MAST_LIST]
BEAM_MAST_LIST_DF2[BEAM_MAST_LIST_DF2.isna().any(axis=1)].to_csv('BEAMS_with_missing_orientation_info.csv')
BEAM_MAST_LIST_DF2.to_csv('BEAMS_comaplete_map.csv')

In [17]:
for key, bm in BEAM_ORIENT.items():
    BEAM_ORIENT[key]['elements'] = sorted(set(BEAM_ORIENT[key]['elements']))
for key, bm in BEAM_ORIENT_ONLY_BEAM.items():
    BEAM_ORIENT_ONLY_BEAM[key]['elements'] = list(BEAM_MAST_LIST_DF2[BEAM_MAST_LIST_DF2['ORIENTATION'] == key].index)

In [18]:
TT = 0
for key, bm in BEAM_ORIENT_ONLY_BEAM.items():
    TT = TT + len(bm['elements'])
print(TT+485, len(BEAM_MAST_LIST))

4513 4513


# Looks Like ORIENTATION IS DEFINE FOR TRUSS ELEMENT ALSO - REMOVING

In [19]:
EXTRA_ORIENTATION = [x for x in list(BEAM_MAST_LIST_DF.index) if x not in BEAM_MAST_LIST]
for eid in EXTRA_ORIENTATION:
    if eid in Spring.keys():
        print(eid,'SPRING')
    elif eid in TRUSS_ELIST:
        print(eid,'TRUSS')
    else:
        print(eid,'SHELL')

8887 TRUSS
8888 TRUSS
8889 TRUSS
8890 TRUSS
8891 TRUSS
8892 TRUSS
8893 TRUSS
8894 TRUSS
8895 TRUSS
8896 TRUSS
8897 TRUSS
8898 TRUSS
8899 TRUSS
8900 TRUSS
8901 TRUSS
8902 TRUSS
8903 TRUSS
8904 TRUSS
8905 TRUSS
8906 TRUSS
8907 TRUSS
8908 TRUSS
8909 TRUSS
8910 TRUSS
8911 TRUSS
8912 TRUSS
8913 TRUSS
8914 TRUSS
8925 TRUSS
8926 TRUSS
8929 TRUSS
8930 TRUSS
8935 TRUSS
8936 TRUSS
8937 TRUSS
8938 TRUSS
9003 TRUSS
9004 TRUSS
9005 TRUSS
9006 TRUSS
9015 TRUSS
9016 TRUSS
9017 TRUSS
9018 TRUSS
9029 TRUSS
9033 TRUSS
9034 TRUSS
9035 TRUSS
9036 TRUSS
9037 TRUSS
9038 TRUSS
9039 TRUSS
9040 TRUSS
9041 TRUSS
9042 TRUSS
9043 TRUSS
9044 TRUSS
9045 TRUSS
9046 TRUSS
9047 TRUSS
9048 TRUSS
9049 TRUSS
9050 TRUSS
9051 TRUSS
9059 TRUSS
9060 TRUSS
9061 TRUSS
9062 TRUSS
9063 TRUSS
9064 TRUSS
9065 TRUSS
9066 TRUSS
9067 TRUSS
9068 TRUSS
9069 TRUSS
9070 TRUSS
9071 TRUSS
9072 TRUSS
9073 TRUSS
9074 TRUSS
9075 TRUSS
9076 TRUSS
9084 TRUSS
9085 TRUSS
9086 TRUSS
9087 TRUSS
9088 TRUSS
9089 TRUSS
9090 TRUSS
9091 TRUSS
9108 TRUSS

In [20]:
for eid in BEAM_MAST_LIST:
    if eid in Spring.keys():
        print(eid,'SPRING')
    elif eid in TRUSS_ELIST:
        print(eid,'TRUSS')
    else:
        pass

In [21]:
with open('BEAM_ORIENT.pickle', 'wb') as handle:
    pickle.dump(BEAM_ORIENT_ONLY_BEAM, handle, protocol=1)
with open('BEAM_WITH_MISSING_ORIENTATION.pickle', 'wb') as handle:
    pickle.dump(list(BEAM_MAST_LIST_DF2[BEAM_MAST_LIST_DF2.isna().any(axis=1)].index), handle, protocol=1)
with open('BEAM_MAST_LIST.pickle', 'wb') as handle:
    pickle.dump(BEAM_MAST_LIST, handle, protocol=1)

In [22]:
with open('BEAM_MAST_LIST_DF.pickle', 'wb') as handle:
    pickle.dump(BEAM_MAST_LIST_DF, handle, protocol=1)
